In [1]:
import keras

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


1D convnets have been used successfully for text classificaton and time series forecasting. They are less computational expensive than RNN. In Keras they can be implemented with the `Conv1D` layer, which takes as input a 3D tensor with shape `(shape, time, features)` and returns similarly shaped 3D tensors. The convolution window is a 1D window on the temporal axis: axis 1 in the input tensor.
We build a simple two-layer 1D convnet and apply it to the IMDB data.

In [3]:
# Let's first load and prepare the data
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000
max_len = 500
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (25000, 500)


In [4]:
# Then let's set up and train the model
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 128)          1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
_________________________________________________________________


Validation accuracy is somewhat less than that of the LSTM, but runtime is faster on both CPU and GPU (the exact increase in speed will vary greatly depending on your exact configuration).

### Combining CNN and RNN for longer sequences

The problem with 1D convnets is that they process input patches independently, so they aren't sensitive to the order of the timesteps, unlike RNN. Of course, to recognize longer-term patterns, you can stack many convo- lution layers and pooling layers, resulting in upper layers that will see long chunks of the original inputs—but that’s still a fairly weak way to induce order sensitivity.

We can see that if we apply a 1D convnet to the temperature forecasting problem, the `MAE` doesn't drop below 0.40 (see example page 229), which is even worse than the non-ML baseline. Because more recent data points should be interpreted differently from older data points in the case of this specific forecasting problem, the convnet fails at producing meaningful results. 

This limitation of convnets isn’t an issue with the IMDB data, because patterns of keywords associated with a positive or negative sentiment are informative independently of where they’re found in the input sentences.

One strategy to combine the speed and lightness of convnets with the order-sensitivity of RNNs is to use a 1D convnet as a preprocessing step before an RNN. Let's try this technique in the temperature forecasting problem.

In [5]:
# Let's first reload the data
import os

data_dir = "/home/ec2-user/datasets/jena_climate/"
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

In [7]:
# Import into Numpy
import numpy as np
float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values
# As a first step, we normalise the data, so that all variables
# are of a similar magnitude
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

In [8]:
# Then we need to write a generator that takes the current array of float data and yields
# batches of data from the recent past, along with a target temperature in the future. It yields a tuple 
# (samples, targets), where samples is one batch of input data and targets is the corresponding array 
# of target temperatures.
def generator(data, lookback, delay, min_index, max_index,
             shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
            
        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows), ))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets    

In [9]:
# Step is now equal to 3, which means we now look at 1 point every 30 minutes. 
# This means we're going to look at twice the data. Lookback and and delay stay the same.
step = 3
lookback = 720
delay = 144

In [10]:
train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step)
val_steps = (300000 - 200001 - lookback) // 128
test_steps = (len(float_data) - 300001 - lookback) // 128

In [12]:
# This is the model, starting with two Conv1D layers and following up with a GRU layer.
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
                       input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 13,697
Trainable params: 13,697
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                                      steps_per_epoch=500,
                                      epochs=20,
                                      validation_data=val_gen,
                                      validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 74s 148ms/step - loss: 0.3300 - val_loss: 0.2818
Epoch 2/20
500/500 [==============================] - 73s 145ms/step - loss: 0.3025 - val_loss: 0.3006
Epoch 3/20
500/500 [==============================] - 72s 145ms/step - loss: 0.2909 - val_loss: 0.2743
Epoch 4/20
500/500 [==============================] - 72s 144ms/step - loss: 0.2818 - val_loss: 0.2698
Epoch 5/20
500/500 [==============================] - 72s 144ms/step - loss: 0.2744 - val_loss: 0.2787
Epoch 6/20
500/500 [==============================] - 72s 145ms/step - loss: 0.2682 - val_loss: 0.2790
Epoch 7/20
500/500 [==============================] - 72s 145ms/step - loss: 0.2596 - val_loss: 0.2795
Epoch 8/20
500/500 [==============================] - 73s 145ms/step - loss: 0.2565 - val_loss: 0.2814
Epoch 9/20
500/500 [==============================] - 73s 145ms/step - loss: 0.2485 - val_loss: 0.2915
Epoch 10/20
500/500 [==============================] - 72s 145ms/step - l

Judging from the validation loss, this setup isn’t as good as the regularized GRU alone, but it’s significantly faster. It looks at twice as much data, which in this case doesn’t appear to be hugely helpful but may be important for other datasets.